## 1. Setup Mountainsort
Before this notebook, need to do 
- (1) Clusterless sorters Notebook 5 in the General Dataprocessing first. That notebook detects lick artifact time.
- (2) decodePrep in Notebook 6. This step unions artifact times detected from the left and right hesmisphere and propagate to every tetrode so that every tetrode has the same artifact times. In addition, this step intersects with the track time obtained from Statescript.)

In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import os
#import cupy as cp
import numpy as np
import datajoint as dj
import spyglass as nd
import pandas as pd
import matplotlib.pyplot as plt
import json
import multiprocessing

# ignore datajoint+jupyter async warnings
import warnings
warnings.simplefilter('ignore', category=DeprecationWarning)
warnings.simplefilter('ignore', category=ResourceWarning)

from spyglass.common import (Session, IntervalList,LabMember, LabTeam, Raw, Session, Nwbfile,
                            Electrode,LFPBand,interval_list_intersect, RawPosition, TaskEpoch)
import spyglass.spikesorting as ss

In [6]:
from spyglass.spikesorting.v0 import (SortGroup, 
                                    SortInterval,
                                    SpikeSortingPreprocessingParameters,
                                    SpikeSortingRecording, 
                                    SpikeSorterParameters,
                                    SpikeSortingRecordingSelection,
                                    ArtifactDetectionParameters, ArtifactDetectionSelection,
                                    ArtifactRemovedIntervalList, ArtifactDetection,
                                      SpikeSortingSelection, SpikeSorting,
                                   CuratedSpikeSortingSelection,CuratedSpikeSorting,Curation)
from spyglass.spikesorting.v0.curation_figurl import CurationFigurl,CurationFigurlSelection
from spyglass.spikesorting.v0.spikesorting_curation import MetricParameters,MetricSelection,QualityMetrics
from spyglass.spikesorting.v0.spikesorting_curation import WaveformParameters,WaveformSelection,Waveforms
from spyglass.utils.nwb_helper_fn import get_nwb_copy_filename
from pprint import pprint

from spyglass.shijiegu.helpers import interval_union
from spyglass.shijiegu.Analysis_SGU import TrialChoice,RippleTimes,EpochPos
from spyglass.shijiegu.load import load_run_sessions
from spyglass.shijiegu.singleUnit import do_mountainSort
from spyglass.shijiegu.decodeHelpers import runSessionNames,sleepSessionNames

### 0. Double check parameter

In [7]:
sorter_params_name = "CA1_tet_Shijie_whiten"
sorter_params = (SpikeSorterParameters & {"sorter_params_name": sorter_params_name}).fetch1("sorter_params")
sorter_params

{'detect_sign': -1,
 'adjacency_radius': -1,
 'freq_min': 0,
 'freq_max': 0,
 'filter': False,
 'whiten': True,
 'num_workers': 4,
 'clip_size': 39,
 'detect_threshold': 3,
 'detect_interval': 10,
 'verbose': True}

### 1. specify data

In [8]:
nwb_copy_file_name = "klein20231107_.nwb"

In [9]:
SortInterval & {'nwb_file_name': nwb_copy_file_name}

nwb_file_name name of the NWB file,sort_interval_name name for this interval,sort_interval 1D numpy array with start and end time for a single interval to be used for spike sorting
klein20231107_.nwb,01_Rev2Sleep1,=BLOB=
klein20231107_.nwb,02_Rev2Session1,=BLOB=
klein20231107_.nwb,03_Rev2Sleep2,=BLOB=
klein20231107_.nwb,04_Rev2Session2,=BLOB=
klein20231107_.nwb,05_Rev2Sleep3,=BLOB=
klein20231107_.nwb,06_Rev2Session3,=BLOB=
klein20231107_.nwb,07_Rev2Sleep4,=BLOB=
klein20231107_.nwb,08_Rev2Session4,=BLOB=
klein20231107_.nwb,09_Rev2Sleep5,=BLOB=
klein20231107_.nwb,10_Rev2Session5,=BLOB=


In [77]:
intervals, _ = runSessionNames(nwb_copy_file_name)
print(intervals)

['02_Rev2Session1', '04_Rev2Session2', '06_Rev2Session3', '08_Rev2Session4', '10_Rev2Session5']


In [78]:
intervals2, _ = sleepSessionNames(nwb_copy_file_name)
print(intervals2)

['01_Rev2Sleep1', '03_Rev2Sleep2', '05_Rev2Sleep3', '07_Rev2Sleep4', '09_Rev2Sleep5', '11_Rev2Sleep6']


### 2. Do sorting

In [79]:
tetrode_with_cell=np.unique((SpikeSortingRecordingSelection & {'nwb_file_name':nwb_copy_file_name}).fetch('sort_group_id'))

In [80]:
tetrode_with_cell

array([  0,  10,  11,  12,  15,  16,  28,  30,  36,  45,  52,  53,  54,
       100, 101])

In [ ]:
for session_name in intervals:
    do_mountainSort(nwb_copy_file_name,session_name)

[15:57:51][INFO] Spyglass: Running spike sorting on {'nwb_file_name': 'klein20231107_.nwb', 'sort_group_id': 0, 'sort_interval_name': '02_Rev2Session1', 'preproc_params_name': 'franklab_tetrode_hippocampus', 'team_name': 'SequenceTask', 'sorter': 'mountainsort4', 'sorter_params_name': 'CA1_tet_Shijie_whiten', 'artifact_removed_interval_list_name': 'klein20231107_.nwb_02_Rev2Session1_0_franklab_tetrode_hippocampus_ampl_1500_prop_075_1ms_artifact_removed_valid_times_track_time_only'}...
INFO:spyglass:Running spike sorting on {'nwb_file_name': 'klein20231107_.nwb', 'sort_group_id': 0, 'sort_interval_name': '02_Rev2Session1', 'preproc_params_name': 'franklab_tetrode_hippocampus', 'team_name': 'SequenceTask', 'sorter': 'mountainsort4', 'sorter_params_name': 'CA1_tet_Shijie_whiten', 'artifact_removed_interval_list_name': 'klein20231107_.nwb_02_Rev2Session1_0_franklab_tetrode_hippocampus_ampl_1500_prop_075_1ms_artifact_removed_valid_times_track_time_only'}...


Mountainsort4 use the OLD spikeextractors mapped with NewToOldRecording
Using temporary directory /stelmo/nwb/tmp/tmpegf305l7
Using 4 workers.
Using tempdir: /stelmo/nwb/tmp/tmpegf305l7/tmp9zhvehub
Num. workers = 4
Preparing /stelmo/nwb/tmp/tmpegf305l7/tmp9zhvehub/timeseries.hdf5...
Preparing neighborhood sorters (M=4, N=75424531)...
Neighboorhood of channel 3 has 4 channels.
Detecting events on channel 4 (phase1)...
Elapsed time for detect on neighborhood: 0:00:10.568933
Num events detected on channel 4 (phase1): 174151
Computing PCA features for channel 4 (phase1)...
Clustering for channel 4 (phase1)...
Found 8 clusters for channel 4 (phase1)...
Computing templates for channel 4 (phase1)...
Re-assigning events for channel 4 (phase1)...
Re-assigning 1 events from 4 to 1 with dt=-1 (k=5)
Re-assigning 16 events from 4 to 3 with dt=-2 (k=8)
Neighboorhood of channel 2 has 4 channels.
Detecting events on channel 3 (phase1)...
Elapsed time for detect on neighborhood: 0:00:07.677975
Num even

/home/shijiegu/anaconda3/envs/spyglass/lib/python3.10/site-packages/spikeinterface/sorters/basesorter.py:254: ResourceWarning: unclosed file <_io.TextIOWrapper name='/stelmo/nwb/recording/klein20231107_.nwb_02_Rev2Session1_0_franklab_tetrode_hippocampus/traces_cached_seg0.raw' mode='r' encoding='UTF-8'>
  SorterClass._run_from_folder(sorter_output_folder, sorter_params, verbose)


mountainsort4 run time 190.50s


[16:01:14][INFO] Spyglass: Saving sorting results...
INFO:spyglass:Saving sorting results...
/home/shijiegu/anaconda3/envs/spyglass/lib/python3.10/site-packages/spikeinterface/core/basesorting.py:237: UserWarning: The registered recording will not be persistent on disk, but only available in memory
  warnings.warn("The registered recording will not be persistent on disk, but only available in memory")
/home/shijiegu/anaconda3/envs/spyglass/lib/python3.10/site-packages/datajoint/autopopulate.py:335: ResourceWarning: unclosed file <_io.TextIOWrapper name='/stelmo/nwb/recording/klein20231107_.nwb_02_Rev2Session1_0_franklab_tetrode_hippocampus/traces_cached_seg0.raw' mode='r' encoding='UTF-8'>
  make(dict(key), **(make_kwargs or {}))
/home/shijiegu/anaconda3/envs/spyglass/lib/python3.10/tempfile.py:869: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/stelmo/nwb/tmp/tmpegf305l7'>
  _warnings.warn(warn_message, ResourceWarning)
[16:01:29][INFO] Spyglass: Running spike sorting o

...

Mountainsort4 use the OLD spikeextractors mapped with NewToOldRecording
Using temporary directory /stelmo/nwb/tmp/tmpjksu7liy
Using 4 workers.
Using tempdir: /stelmo/nwb/tmp/tmpjksu7liy/tmpog_sdhhj
Num. workers = 4
Preparing /stelmo/nwb/tmp/tmpjksu7liy/tmpog_sdhhj/timeseries.hdf5...
Preparing neighborhood sorters (M=4, N=75201920)...
Neighboorhood of channel 3 has 4 channels.
Detecting events on channel 4 (phase1)...
Elapsed time for detect on neighborhood: 0:00:07.331834
Num events detected on channel 4 (phase1): 99390
Computing PCA features for channel 4 (phase1)...
Clustering for channel 4 (phase1)...
Found 1 clusters for channel 4 (phase1)...
Computing templates for channel 4 (phase1)...
Re-assigning events for channel 4 (phase1)...
Neighboorhood of channel 2 has 4 channels.
Detecting events on channel 3 (phase1)...
Elapsed time for detect on neighborhood: 0:00:07.671651
Num events detected on channel 3 (phase1): 107797
Computing PCA features for channel 3 (phase1)...
Clustering fo

/home/shijiegu/anaconda3/envs/spyglass/lib/python3.10/site-packages/spikeinterface/sorters/basesorter.py:254: ResourceWarning: unclosed file <_io.TextIOWrapper name='/stelmo/nwb/recording/klein20231107_.nwb_10_Rev2Session5_36_franklab_tetrode_hippocampus/traces_cached_seg0.raw' mode='r' encoding='UTF-8'>
  SorterClass._run_from_folder(sorter_output_folder, sorter_params, verbose)


mountainsort4 run time 125.20s


[18:17:21][INFO] Spyglass: Saving sorting results...
INFO:spyglass:Saving sorting results...
/home/shijiegu/anaconda3/envs/spyglass/lib/python3.10/site-packages/spikeinterface/core/basesorting.py:237: UserWarning: The registered recording will not be persistent on disk, but only available in memory
  warnings.warn("The registered recording will not be persistent on disk, but only available in memory")
/home/shijiegu/anaconda3/envs/spyglass/lib/python3.10/site-packages/datajoint/autopopulate.py:335: ResourceWarning: unclosed file <_io.TextIOWrapper name='/stelmo/nwb/recording/klein20231107_.nwb_10_Rev2Session5_36_franklab_tetrode_hippocampus/traces_cached_seg0.raw' mode='r' encoding='UTF-8'>
  make(dict(key), **(make_kwargs or {}))
/home/shijiegu/anaconda3/envs/spyglass/lib/python3.10/tempfile.py:869: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/stelmo/nwb/tmp/tmpjksu7liy'>
  _warnings.warn(warn_message, ResourceWarning)
[18:17:37][INFO] Spyglass: Running spike sorting 

Mountainsort4 use the OLD spikeextractors mapped with NewToOldRecording
Using temporary directory /stelmo/nwb/tmp/tmpuyjf28um
Using 4 workers.
Using tempdir: /stelmo/nwb/tmp/tmpuyjf28um/tmpy32_qyz9
Num. workers = 4
Preparing /stelmo/nwb/tmp/tmpuyjf28um/tmpy32_qyz9/timeseries.hdf5...
Preparing neighborhood sorters (M=4, N=75201920)...
Neighboorhood of channel 3 has 4 channels.
Detecting events on channel 4 (phase1)...
Elapsed time for detect on neighborhood: 0:00:08.430898
Num events detected on channel 4 (phase1): 296383
Computing PCA features for channel 4 (phase1)...
Clustering for channel 4 (phase1)...
Found 10 clusters for channel 4 (phase1)...
Computing templates for channel 4 (phase1)...
Re-assigning events for channel 4 (phase1)...
Re-assigning 180 events from 4 to 1 with dt=-2 (k=3)
Neighboorhood of channel 0 has 4 channels.
Detecting events on channel 1 (phase1)...


In [32]:
for session_name in intervals2:
    do_mountainSort(nwb_copy_file_name,session_name)

In [12]:
all_tet_list = np.unique((SpikeSorting & {"nwb_file_name": nwb_copy_file_name,"sorter":"mountainsort4"}).fetch("sort_group_id"))

In [13]:
for tet in all_tet_list:
    key = {'nwb_file_name' : nwb_copy_file_name,
           "sort_group_id": tet,"sorter":"mountainsort4"}
    assert len(SpikeSortingRecordingSelection & key) == len(SpikeSortingRecording & key)
    
(SpikeSorting & {"nwb_file_name": nwb_copy_file_name,"sorter":"mountainsort4"})

nwb_file_name name of the NWB file,sort_group_id identifier for a group of electrodes,sort_interval_name name for this interval,preproc_params_name,team_name,sorter,sorter_params_name,artifact_removed_interval_list_name,sorting_path,"time_of_sort in Unix time, to the nearest second"
klein20231107_.nwb,0,02_Rev2Session1,franklab_tetrode_hippocampus,SequenceTask,mountainsort4,CA1_tet_Shijie_whiten,klein20231107_.nwb_02_Rev2Session1_0_franklab_tetrode_hippocampus_ampl_1500_prop_075_1ms_artifact_removed_valid_times_track_time_only,/stelmo/nwb/sorting/klein20231107_.nwb_02_Rev2Session1_0_franklab_tetrode_hippocampus_2279da71_spikesorting,1749250874
klein20231107_.nwb,0,04_Rev2Session2,franklab_tetrode_hippocampus,SequenceTask,mountainsort4,CA1_tet_Shijie_whiten,klein20231107_.nwb_04_Rev2Session2_0_franklab_tetrode_hippocampus_ampl_1500_prop_075_1ms_artifact_removed_valid_times_track_time_only,/stelmo/nwb/sorting/klein20231107_.nwb_04_Rev2Session2_0_franklab_tetrode_hippocampus_4e4d93ce_spikesorting,1749252535
klein20231107_.nwb,0,06_Rev2Session3,franklab_tetrode_hippocampus,SequenceTask,mountainsort4,CA1_tet_Shijie_whiten,klein20231107_.nwb_06_Rev2Session3_0_franklab_tetrode_hippocampus_ampl_1500_prop_075_1ms_artifact_removed_valid_times_track_time_only,/stelmo/nwb/sorting/klein20231107_.nwb_06_Rev2Session3_0_franklab_tetrode_hippocampus_5f41440e_spikesorting,1749253932
klein20231107_.nwb,0,08_Rev2Session4,franklab_tetrode_hippocampus,SequenceTask,mountainsort4,CA1_tet_Shijie_whiten,klein20231107_.nwb_08_Rev2Session4_0_franklab_tetrode_hippocampus_ampl_1500_prop_075_1ms_artifact_removed_valid_times_track_time_only,/stelmo/nwb/sorting/klein20231107_.nwb_08_Rev2Session4_0_franklab_tetrode_hippocampus_d6519b14_spikesorting,1749255788
klein20231107_.nwb,0,10_Rev2Session5,franklab_tetrode_hippocampus,SequenceTask,mountainsort4,CA1_tet_Shijie_whiten,klein20231107_.nwb_10_Rev2Session5_0_franklab_tetrode_hippocampus_ampl_1500_prop_075_1ms_artifact_removed_valid_times_track_time_only,/stelmo/nwb/sorting/klein20231107_.nwb_10_Rev2Session5_0_franklab_tetrode_hippocampus_6d4c7034_spikesorting,1749257756
klein20231107_.nwb,10,02_Rev2Session1,franklab_tetrode_hippocampus,SequenceTask,mountainsort4,CA1_tet_Shijie_whiten,klein20231107_.nwb_02_Rev2Session1_10_franklab_tetrode_hippocampus_ampl_1500_prop_075_1ms_artifact_removed_valid_times_track_time_only,/stelmo/nwb/sorting/klein20231107_.nwb_02_Rev2Session1_10_franklab_tetrode_hippocampus_bcf4064b_spikesorting,1749250994
klein20231107_.nwb,10,04_Rev2Session2,franklab_tetrode_hippocampus,SequenceTask,mountainsort4,CA1_tet_Shijie_whiten,klein20231107_.nwb_04_Rev2Session2_10_franklab_tetrode_hippocampus_ampl_1500_prop_075_1ms_artifact_removed_valid_times_track_time_only,/stelmo/nwb/sorting/klein20231107_.nwb_04_Rev2Session2_10_franklab_tetrode_hippocampus_8bba543f_spikesorting,1749252629
klein20231107_.nwb,10,06_Rev2Session3,franklab_tetrode_hippocampus,SequenceTask,mountainsort4,CA1_tet_Shijie_whiten,klein20231107_.nwb_06_Rev2Session3_10_franklab_tetrode_hippocampus_ampl_1500_prop_075_1ms_artifact_removed_valid_times_track_time_only,/stelmo/nwb/sorting/klein20231107_.nwb_06_Rev2Session3_10_franklab_tetrode_hippocampus_4356a485_spikesorting,1749254063
klein20231107_.nwb,10,08_Rev2Session4,franklab_tetrode_hippocampus,SequenceTask,mountainsort4,CA1_tet_Shijie_whiten,klein20231107_.nwb_08_Rev2Session4_10_franklab_tetrode_hippocampus_ampl_1500_prop_075_1ms_artifact_removed_valid_times_track_time_only,/stelmo/nwb/sorting/klein20231107_.nwb_08_Rev2Session4_10_franklab_tetrode_hippocampus_4e9a79ba_spikesorting,1749255931
klein20231107_.nwb,10,10_Rev2Session5,franklab_tetrode_hippocampus,SequenceTask,mountainsort4,CA1_tet_Shijie_whiten,klein20231107_.nwb_10_Rev2Session5_10_franklab_tetrode_hippocampus_ampl_1500_prop_075_1ms_artifact_removed_valid_times_track_time_only,/stelmo/nwb/sorting/klein20231107_.nwb_10_Rev2Session5_10_franklab_tetrode_hippocampus_692b19dc_spikesorting,1749257887


In [14]:
(SpikeSorting & {"nwb_file_name": nwb_copy_file_name,"sorter":"mountainsort4"}).fetch("sort_interval_name")

array(['02_Rev2Session1', '04_Rev2Session2', '06_Rev2Session3',
       '08_Rev2Session4', '10_Rev2Session5', '02_Rev2Session1',
       '04_Rev2Session2', '06_Rev2Session3', '08_Rev2Session4',
       '10_Rev2Session5', '02_Rev2Session1', '04_Rev2Session2',
       '06_Rev2Session3', '08_Rev2Session4', '10_Rev2Session5',
       '02_Rev2Session1', '04_Rev2Session2', '06_Rev2Session3',
       '08_Rev2Session4', '10_Rev2Session5', '02_Rev2Session1',
       '04_Rev2Session2', '06_Rev2Session3', '08_Rev2Session4',
       '10_Rev2Session5', '02_Rev2Session1', '04_Rev2Session2',
       '06_Rev2Session3', '08_Rev2Session4', '10_Rev2Session5',
       '02_Rev2Session1', '04_Rev2Session2', '06_Rev2Session3',
       '08_Rev2Session4', '10_Rev2Session5', '02_Rev2Session1',
       '04_Rev2Session2', '06_Rev2Session3', '08_Rev2Session4',
       '10_Rev2Session5', '02_Rev2Session1', '04_Rev2Session2',
       '06_Rev2Session3', '08_Rev2Session4', '10_Rev2Session5',
       '02_Rev2Session1', '04_Rev2Sessio